In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

print("ok")

ok


In [0]:
url="https://raw.githubusercontent.com/epfml/ML_course/master/labs/ex07/template/data/train.csv"
train=pd.read_csv(url)

#to see properly all the columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

train.head

<bound method NDFrame.head of             Id Prediction  DER_mass_MMC  DER_mass_transverse_met_lep  DER_mass_vis  DER_pt_h  DER_deltaeta_jet_jet  DER_mass_jet_jet  DER_prodeta_jet_jet  DER_deltar_tau_lep  DER_pt_tot  DER_sum_pt  DER_pt_ratio_lep_tau  DER_met_phi_centrality  DER_lep_eta_centrality  PRI_tau_pt  PRI_tau_eta  PRI_tau_phi  PRI_lep_pt  PRI_lep_eta  PRI_lep_phi  PRI_met  PRI_met_phi  PRI_met_sumet  PRI_jet_num  PRI_jet_leading_pt  PRI_jet_leading_eta  PRI_jet_leading_phi  PRI_jet_subleading_pt  PRI_jet_subleading_eta  PRI_jet_subleading_phi  PRI_jet_all_pt
0       100000  s          138.470       51.655                       97.827        27.980    0.910                 124.711           2.666                3.064               41.928      197.760     1.582                 1.396                   0.200                   32.638      1.017        0.381        51.626      2.273       -2.414        16.824  -0.277        258.733        2            67.435              2.150       


# Helper functions
---



In [0]:
def load_prepare_data(data):
  y=data.iloc[:,1]
  x=data.iloc[:,2:]
  
  #transform b in -1 and s in 1
  
  yb=np.ones(y.shape[0])
  yb[np.where(y=='b')]=-1
  
  return yb,x
  

In [0]:
def standardize(train):
  mean_train=np.mean(train)
  std_train=np.std(train)
  
  train-=mean_train
  train=train/std_train
  
  return train
  
 

In [0]:
def predict_labels(weights,data):
  predictions=data.dot(weights)
  predictions[np.where(predictions<=0)]=-1
  predictions[np.where(predictions>0)]=1
  
  return predictions

In [0]:
def transform_x(x):
  tx=np.c_[np.ones(x.shape[0]),x]
  return tx

In [0]:
def calculate_accuracy(y,tx,w):
  predictions=predict_labels(w,tx)
  print(predictions)
  hit_mark=0
  for i,pred in enumerate(predictions):
    if pred==y[i]:
      hit_mark+=1
  
  return hit_mark/y.shape[0]


In [0]:
def calculate_primal_objective(y,tx,w,lambda_):
  #calculate the loss of the real loss function, this might not be as useful as the accuracy for a boolean classification
  elements=np.maximum(np.zeros(y.shape[0]),1-y*(np.dot(tx,w)))
  loss=np.sum(elements)+lambda_*(np.dot(w,w))*0.5
  
  return loss

**Exercice 1: Stochastic Gradient Descent for SVM (Ridge)**

In [0]:
def calculate_stochastic_gradient(y,tx,w,lambda_,n):
  #n represents a random n for element wise stochastic gradient descent
  N=tx.shape[0]
  x_n,y_n=tx[n],y[n]
  if (1-y_n*(x_n.T).dot(w))>0:
    return -y_n*x_n+lambda_*w
  else:
    return 0+lambda_*w
  

In [0]:
def sgd_for_svm_demo(y,tx):
  max_iter= 100000
  gamma=0.001
  lambda_=0.1

  w=np.random.rand(tx.shape[1])
  for i in range(max_iter):
    n=np.random.randint(0,tx.shape[0])
    gradient=calculate_stochastic_gradient(y,tx,w,lambda_,n)
    w-=gamma*gradient
    
    if i%10000==0:
      print("iteration={it}, cost={c}".format(it=i,c=calculate_primal_objective(y,tx,w,lambda_)))
    
  print("trainning accuracy is, for the last loss,  ", calculate_accuracy(y,tx,w)*100, "%")
 
    

In [0]:
y,x=load_prepare_data(train)
x=standardize(x)
tx=transform_x(x)

index = np.random.choice(tx.shape[0], 250, replace=False)  
tx=tx[index]
y=y[index]
               

In [0]:
sgd_for_svm_demo(y,tx)

iteration=0, cost=733.8990472885556
iteration=10000, cost=136.8768052944035
iteration=20000, cost=134.82533551569682
iteration=30000, cost=132.23140710061182
iteration=40000, cost=133.1726425879873
iteration=50000, cost=132.2882312813044
iteration=60000, cost=134.03217710030447
iteration=70000, cost=133.12682775397334
iteration=80000, cost=132.29656921070506
iteration=90000, cost=132.53203395142938
[-1. -1. -1. -1.  1. -1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1. -1. -1.
  1. -1.  1. -1. -1. -1.  1.  1. -1. -1.  1. -1.  1. -1. -1.  1.  1.  1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.
 -1.  1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1.
 -1. -1.  1. -1.  1. -1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1. -1.
  1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1.  1.  1.  1. -1. -1. -1.  1.
 -1. -1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
 -1.  1. -1. -

Prediction seems highly variant without ridge, on the other hand with ridge it's way better

# **Coordinate descent for dual representation of svm**

In [0]:
def calculate_coordinate_update(y,X,lambda_,alpha,w,n):
  #n represents the randomly chosen coordinate. For every exemple there's one alpha
  x_n,y_n=X[n],y[n]
  old_alpha_n=np.copy(alpha[n])
  g=y_n*x_n.dot(w)-1
  if old_alpha_n==0:
    g=min(0,g)
  elif old_alpha_n==1.0:
    g=max(0,g)
  else:
    g=g
  if g!=0:
    alpha[n]=min(max(old_alpha_n- lambda_ * g / (x_n.T.dot(x_n)),0),1)
    
    w+=1.0/lambda_*(alpha[n]-old_alpha_n)*x_n*y_n
  return w,alpha
  

In [0]:
def calculate_dual_objective(y,X,w,alpha,lambda_):
  return np.sum(alpha)- lambda_ / 2.0 * np.sum(w ** 2)


In [0]:
def coordinate_descent_for_svm_demo(y,X):
  max_iter=100000
  lambda_=0.001
  w=np.zeros(X.shape[1])
  alpha=np.zeros(X.shape[0])
  
  for it in range(max_iter):
    n=np.random.randint(0,X.shape[0])
    w,alpha=calculate_coordinate_update(y,X,lambda_,alpha,w,n)
    if it % 10000 == 0:
      # primal objective
      primal_value = calculate_primal_objective(y, X, w, lambda_)
      # dual objective
      dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
      # primal dual gap
      duality_gap = primal_value - dual_value
      print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
  print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, tx)

iteration=0, primal:259.53998, dual:0.00001, gap:259.53996
iteration=10000, primal:193.29785, dual:0.23251, gap:193.06533
iteration=20000, primal:163.07525, dual:0.46674, gap:162.60851
iteration=30000, primal:168.29330, dual:0.69040, gap:167.60290
iteration=40000, primal:175.45993, dual:0.91680, gap:174.54314
iteration=50000, primal:171.60973, dual:1.14910, gap:170.46062
iteration=60000, primal:208.13994, dual:1.38100, gap:206.75893
iteration=70000, primal:162.02056, dual:1.61138, gap:160.40919
iteration=80000, primal:171.26260, dual:1.83554, gap:169.42706
iteration=90000, primal:175.09893, dual:2.07378, gap:173.02515
[-1. -1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1. -1.
 -1. -1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1. -1.  1.
 -1. -1.  1.  1.  1. -1. -1. -1.  1.  1. -1. -1. -1.  1.  1. -1.  1.  1.
  1.  1.  1.  1. -1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1.  1.  1.  1. -1.  1.  1. -1. -1. -1.  1.  1.
 -1. -1. 

 **Extension: trying Kernel trick with svm**

In [0]:
def kernel_function(xi,xj,gamma):
  return (1+xj.T.dot(xi))*3
  #return np.exp(-gamma*((xi-xj).dot((xi-xj))))
  

In [0]:
def calculate_kernel_matrix(X,gamma):
  matrix=np.zeros((X.shape[0],X.shape[0]))
  for i in range(X.shape[0]):
    for j in range(X.shape[0]):
      matrix[i][j]=kernel_function(X[i],X[j],gamma)
  print("matrix",matrix)
  return matrix
      

In [0]:
def predict_label_kernel(alphas,kernel_matrix):
  predictions=kernel_matrix.dot(alphas)
  
  print("hello",predictions)
  predictions[np.where(predictions<=0)]=-1
  predictions[np.where(predictions>0)]=1
  
  return predictions
  
  

In [0]:
def calculate_Q_matrix(y,kernel_matrix):
  return y*kernel_matrix*y

In [0]:
def calculate_dual_loss_kernel(Q,lambda_,alpha):
  return np.sum(alpha)-0.5*lambda_*alpha.T.dot(Q).dot(alpha)

  

In [0]:
def calculate_coordinate_update_kernel(y,Q,lambda_,alpha,n):
  sum_=0
  for i in range(Q.shape[0]):
    if i!=n:
      sum_+=Q[n][i]*alpha[i]
  
  alpha[n]=1/Q[n][n]*(lambda_-sum_)
  
  return alpha
 
  
  

In [0]:
def coordinate_descent_for_svm_kernel_demo(y,X):
  gamma=0.001
  kernel_matrix=calculate_kernel_matrix(X,gamma)
  Q=calculate_Q_matrix(y,kernel_matrix)
  
  max_iter=100000
  lambda_=1
  alpha=np.zeros(X.shape[0])
  
  for it in range(max_iter):
    n=np.random.randint(0,X.shape[0])
    alpha=calculate_coordinate_update_kernel(y,Q,lambda_,alpha,n)
    
    if it%10000==0: 
      print(calculate_dual_loss_kernel(Q,lambda_,alpha))
  print("alpha",alpha)
  print(calculate_accuracy_kernel(y,kernel_matrix,alpha))
    
  
    
    
  

In [0]:
def calculate_accuracy_kernel(y,kernel_matrix,alpha):
  predictions=predict_label_kernel(alpha,kernel_matrix)
  print(predictions)
  hit_mark=0
  for i,pred in enumerate(predictions):
    if pred==y[i]:
      hit_mark+=1
  
  return hit_mark/y.shape[0]
  

In [0]:
print(tx.shape)

(250, 31)


In [0]:
coordinate_descent_for_svm_kernel_demo(y,tx)

matrix [[ 92.48055038 -64.23055632  11.25427716 ...  37.73037253  40.3522982
  -15.70467469]
 [-64.23055632 161.31049978 -10.50998203 ... -31.22095883 -53.67891938
   -7.95313923]
 [ 11.25427716 -10.50998203 196.08707171 ... -22.67982363  -1.90485337
   35.54617752]
 ...
 [ 37.73037253 -31.22095883 -22.67982363 ...  77.1889438   41.78080232
   10.10263012]
 [ 40.3522982  -53.67891938  -1.90485337 ...  41.78080232  57.28339742
   10.53129561]
 [-15.70467469  -7.95313923  35.54617752 ...  10.10263012  10.53129561
   52.05776054]]
0.006790663005131083
0.08333272956379785
0.08333285859680152
0.08333296266015734
0.08333303284610516
0.08333308719878589
0.08333312797964477
0.08333315961465838
0.08333318254218913
0.08333320064244143
alpha [-7.70385253e-04 -4.69420927e-04  1.08848181e-04  3.05216890e-04
  2.07994408e-03  1.39861024e-03 -8.63161061e-04  7.36329957e-05
 -3.85281490e-06 -2.81947371e-04  3.80627035e-04  1.99571848e-04
  5.93340939e-04  7.29971144e-05  3.53409655e-03  1.19602860e-03